<h1>Segmenting and Clustering Chinese Restaurants in Toronto<h1/>

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>


Cluster Toronto areas with respect to the cost of meals in restaurants.


## Data <a name="data"></a>

In [1]:
import pandas as pd 
import numpy as np
import requests

Preparing dataframe with Toronto Postal code adresses, Neighbourhoods, Lattitude and Longitude.

In [2]:
# url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
# wiki_df = pd.read_html(requests.get(url).text)[0]
# wiki_df = wiki_df[wiki_df['Borough'] != 'Not assigned']
# coord_df = pd.read_csv('Geospatial_Coordinates.csv')
# wiki_df = wiki_df.merge(coord_df, how='left', on='Postal Code')
# wiki_df.head()

Using Folium library to visualise postal code areas

In [3]:
# import folium

# toronto_map = folium.Map(location=[43.7, -79.4], zoom_start=11)
# toronto_map.choropleth(
#     geo_data='Toronto2.geojson',
#     fill_color='Yellow', 
#     line_color='Red',
#     fill_opacity=0.7, 
#     line_opacity=1,
#     reset=True
# )

# postcodes = folium.map.FeatureGroup()

# for lat, lng, in zip(wiki_df.Latitude, wiki_df.Longitude):
#     postcodes.add_child(
#         folium.features.CircleMarker(
#             [lat, lng],
#             radius=5, # define how big you want the circle markers to be
#             color='yellow',
#             fill=True,
#             fill_color='blue',
#             fill_opacity=0.6
#         )
#     )

# # add pop-up text to each marker on the map
# latitudes = list(wiki_df.Latitude)
# longitudes = list(wiki_df.Longitude)
# labels = list(wiki_df['Postal Code'])

# for lat, lng, label in zip(latitudes, longitudes, labels):
#     folium.Marker([lat, lng], popup=label).add_to(toronto_map)    
    
# # add incidents to map
# toronto_map.add_child(postcodes)

Using Foursquare API to collect information about chinese restaurants in every area 

In [4]:
# CLIENT_ID = 'ULGA5LTQONIWZ5ULCDROWAYVYJ0ZOI0C35CLVMG12JJAKCFN' 
# CLIENT_SECRET = 'VMEKZXMMCBO3RWK0IFRKV5KXBABWQ1EQ3J2ACE53UAR5SVWC'
# VERSION = '20180604'
# LIMIT = 50
# search_query = 'vietnamese'# tested: chinese, asian, korean, japanese, vietnamese
# radius = 1000



function needs to be written that converts <b>result</b> request and transform it into the final desired dataframe with all the neccessary information 

Dataframe to have the following columns: <br> 

|Columns      |   
|----------   |
|Venue        |
|Venue id     | 
|Toronto area |  
|Category.name| 
|Address      |
|Longitude    |
|Lattitude    |
""

In [ ]:
"""  
loop through all the types of restaurants 
"""

In [5]:
# unprocessed_df = pd.DataFrame()

# toronto_postal_code = wiki_df['Postal Code'].tolist()
# toronto_borough = wiki_df['Borough'].tolist()

# list_of_df = []
# for postcode, borough in zip(toronto_postal_code, toronto_borough): 
#     near = " ".join([borough, postcode])
#     url =   f'https://api.foursquare.com/v2/venues/'\
#             f'search?client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}'\
#             f'&near={near}&v={VERSION}&query={search_query}&limit={LIMIT}'

#     try: 
#         results = requests.get(url).json()
#         temp_df = pd.DataFrame(results['response']['venues'])
#         list_of_df.append(temp_df)
#     except BaseException:
#         print(f'no result for: {near}')

In [6]:
#concatinating all the dataframes and saving to csv file for future use 
# unprocessed_df = pd.concat(list_of_df, ignore_index=True)
# unprocessed_df = unprocessed_df.drop_duplicates(subset='name')
# unprocessed_df.to_csv('vietnamese_unprocessed.csv')
# unprocessed_df.shape

Processing dataframes obtained above 

In [73]:
def unpack_name_categories(row):
    row = eval(row)
    if len(row) == 0:
        return np.NaN
    else:
        return row[0]['name'] 

def unpack_shortName_categories(row):
    row = eval(row)
    if len(row) == 0:
        return np.NaN
    else:
        return row[0]['shortName'] 


def unpack_address_location(row):
    row = eval(row)
    if row:
        try:
            return row['address']
        except BaseException:
            return np.NaN
    else: 
        return np.NaN

def unpack_lat_location(row):
    row = eval(row)
    if row:
        return row['lat']
    else: 
        return np.NaN

def unpack_lng_location(row):
    row = eval(row)
    if row:
        try:
            return row['lng']
        except BaseException:
            return np.NaN
    else: 
        return np.NaN

def unpack_postcode_location(row):
    row = eval(row)
    if row:
        try:
            return row['postalCode']
        except BaseException:
            return np.NaN
    else: 
        return np.NaN

In [40]:
unprocessed_df = pd.read_csv('all-restaurants-unprocessed.csv')

In [41]:
#creating a template for final results of SEARCH query 
columns = ['VenueID', 'Venue', 'CategoryName', 'CategoryShortName', 'Address', 'PostCode', 'Longitude', 'Lattitude']
restaurants_df = pd.DataFrame(columns=columns)

In [116]:
#writing a function that extract needed keys and values from categories column 
restaurants_df.VenueID = unprocessed_df.id
restaurants_df.Venue = unprocessed_df.name
restaurants_df.CategoryName = unprocessed_df.apply(lambda x: unpack_name_categories(x.categories), axis=1)
restaurants_df.CategoryShortName = unprocessed_df.apply(lambda x: unpack_shortName_categories(x.categories), axis=1)
restaurants_df.Address = unprocessed_df.apply(lambda x: unpack_address_location(x.location), axis=1)
restaurants_df.PostCode = unprocessed_df.apply(lambda x: unpack_postcode_location(x.location), axis=1)
restaurants_df.Longitude = unprocessed_df.apply(lambda x: unpack_lng_location(x.location), axis=1)
restaurants_df.Lattitude = unprocessed_df.apply(lambda x: unpack_lat_location(x.location), axis=1)

In [142]:
true_false = restaurants_df.CategoryShortName.value_counts() < 10
true_false.value_counts()

True     87
False     7
Name: CategoryShortName, dtype: int64

short name in informative <br>
record postcode values that were searched <br>
investigate what keys location and categories column returns <br>
Category IDs corresponding to Italian restaurants were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):
<br>
split toronto into large number of circles and find restaurants from these circles <br> 

